In [42]:
import os
import sys
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
from pprint import pprint
import torch
sys.path.append('..')
sys.path.append('/home/valerie/Projects/Alps_LCC/models')
print(sys.path)

import torch
from torchvision.models import resnet50
import matplotlib.pyplot as plt
from models.ACE_Better_DeepLabV3Plus import ACE_deeplabv3P_w_Better_Experts
from argparse import Namespace
from dataset.SwissImageDataset import SwissImage
from torch.utils.data import DataLoader
torch.set_grad_enabled(False) 
from torch.nn.functional import softmax


['/home/valerie/Projects/Alps_LCC/notebook', '/home/valerie/anaconda3/envs/ace/lib/python311.zip', '/home/valerie/anaconda3/envs/ace/lib/python3.11', '/home/valerie/anaconda3/envs/ace/lib/python3.11/lib-dynload', '', '/home/valerie/anaconda3/envs/ace/lib/python3.11/site-packages', '..', '/home/valerie/Projects/Alps_LCC/models', '.', '.', 'models/', '.', '..', '..', '/home/valerie/Projects/Alps_LCC/models', '..', '/home/valerie/Projects/Alps_LCC/models']


In [43]:
classes = {'Background':0, "Bedrock" : 1, "Bedrockwith grass" : 2,
                    "Large blocks" : 3, "Large blocks with grass" : 4, "Scree" : 5,
                    "Scree with grass" : 6,"Water" : 7,
                    "Forest" : 8, "Glacier" : 9, }  

In [44]:
config_fp ='/home/valerie/Projects/Alps_LCC/out/experts/ace_3exp_30June/config.json'
checkpoint_path = '/home/valerie/Projects/Alps_LCC/out/experts/ace_3exp_30June/current_best.pt'
# Open the JSON file and load its contents as a dictionary
with open(config_fp, 'r') as json_file:
    cfg = json.load(json_file)
    args = Namespace(**cfg)
    pprint(cfg)
checkpoint = torch.load(checkpoint_path)
pprint(checkpoint.keys())

{'CNN_aggregator': False,
 'L2penalty': False,
 'MLP_aggregator': False,
 'bs': 64,
 'debug': False,
 'device': 'cuda',
 'epoch': 50,
 'experts': 3,
 'force_cpu': False,
 'log_wandb': True,
 'loss': 'celoss',
 'lr': 1e-05,
 'lr_decay_rate': 0.1,
 'lws': False,
 'model': 'Deeplabv3_w_Better_Experts',
 'name': 'ace_3exp_30June',
 'num_workers': 16,
 'out_dir': 'out/baseline/',
 'seed': 2436,
 'separate_backprop': False,
 'small_dataset': True,
 'step_size': 10,
 'test_only': False,
 'weight_decay': 0.01}
dict_keys(['epoch', 'state_dict', 'perf', 'last_epoch', 'optimizer'])


In [45]:
model = ACE_deeplabv3P_w_Better_Experts(num_classes=10,num_experts=args.experts, pretrained_backbone=True,use_lws=True,)
model.load_state_dict(checkpoint['state_dict'])
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device
model = model.to(device)

In [48]:
img_dir = '/home/valerie/data/rocky_tlm/rgb/' 
dem_dir = '/home/valerie/data/rocky_tlm/dem/' 
mask_dir = '/home/valerie/data/ace_alps/mask' 
test_csv = '/home/valerie/Projects/Alps_LCC/data/split/test_dataset.csv'


test_dataset = SwissImage(
    dataset_csv = test_csv,
    img_dir = img_dir,
    dem_dir = dem_dir,
    mask_dir = mask_dir,
    common_transform=None,
    img_transform= None,
    debug= False,          
)
test_loader = DataLoader(test_dataset, 
    batch_size= 8,
    shuffle=False,
    num_workers= 8,
    pin_memory=True
)



In [49]:
from utils.inference_utils import MetricLogger
metrics_0 = MetricLogger( n_classes=10)
metrics_1 = MetricLogger( n_classes=10)
metrics_2 = MetricLogger(10)

for i, (image, dem, mask) in tqdm(enumerate(test_loader)):
    
    # move data to device
    input = torch.cat((image, dem), dim=1).to(device) 
    mask = mask.to(device) 
    num_inputs = input.size(0)
    
    # Run forwards pass and compute loss : 
    output = model(input) 
    
    exp_0_logits =  softmax( output['exp_0'],dim=1)
    preds_0 = torch.argmax(exp_0_logits.detach().cpu(),axis=1) 
    exp_1_logits =  softmax(output['exp_1'], dim=1 )  
    preds_1 = torch.argmax(exp_1_logits.detach().cpu(),axis=1) 
    exp_2_logits= softmax( output['exp_2'],dim=1)
    preds_2 = torch.argmax(exp_2_logits.detach().cpu(),axis=1) 
    
    gt = mask.squeeze().detach().cpu().numpy()
    metrics_0.update(gt, preds_0.numpy())
    metrics_1.update(gt, preds_1.numpy())
    metrics_2.update(gt, preds_2.numpy())

5739it [10:28,  9.13it/s]


In [50]:
mean_acc, mean_iou, acc_cls, overall_acc = metrics_0.get_scores()
print('mean_acc, mean_iou, overall_acc ')
print('expert 0' )
class_accuracies = { cls : np.round (value,3) for cls, value in zip (classes.keys(),acc_cls )  }     
freq_cls_acc =   1/4* (class_accuracies["Scree"]+ class_accuracies["Bedrock"] + class_accuracies["Glacier"] + class_accuracies["Forest"])
common_cls_acc = 1/3* (class_accuracies["Scree with grass"]+    class_accuracies["Water"]+   class_accuracies["Bedrockwith grass"])
rare_cls_acc =   1/2* (class_accuracies["Large blocks"]+   class_accuracies["Large blocks with grass"]) 
metrics = {
            'test_miou' : np.round ( mean_iou,3), 
            'test_macc':  np.round (mean_acc,3), 
            'test_oacc':  np.round (overall_acc,3),
            'frequent_cls_acc':np.round(freq_cls_acc,3),
            'common_cls_acc': np.round(common_cls_acc,3),
            'rare_cls_acc': np.round(rare_cls_acc,3),
            'test_acc' :  class_accuracies,
    }
pprint(metrics)
#
mean_acc, mean_iou, acc_cls, overall_acc = metrics_1.get_scores()
print('expert 1' )
class_accuracies = { cls : np.round (value,3) for cls, value in zip (classes.keys(),acc_cls )  }
freq_cls_acc =   1/4* (class_accuracies["Scree"]+ class_accuracies["Bedrock"] + class_accuracies["Glacier"] + class_accuracies["Forest"])
common_cls_acc = 1/3* (class_accuracies["Scree with grass"]+    class_accuracies["Water"]+   class_accuracies["Bedrockwith grass"])
rare_cls_acc =   1/2* (class_accuracies["Large blocks"]+   class_accuracies["Large blocks with grass"])
metrics = {
            'test_miou' : np.round ( mean_iou,3), 
            'test_macc':  np.round (mean_acc,3), 
            'test_oacc':  np.round (overall_acc,3),
            'frequent_cls_acc':np.round(freq_cls_acc,3),
            'common_cls_acc': np.round(common_cls_acc,3),
            'rare_cls_acc': np.round(rare_cls_acc,3),
            'test_acc' :  class_accuracies,
    }
pprint(metrics)
mean_acc, mean_iou, acc_cls, overall_acc = metrics_2.get_scores()
print('expert 2' )
class_accuracies = { cls : np.round (value,3) for cls, value in zip (classes.keys(),acc_cls )  }
freq_cls_acc =   1/4* (class_accuracies["Scree"]+ class_accuracies["Bedrock"] + class_accuracies["Glacier"] + class_accuracies["Forest"])
common_cls_acc = 1/3* (class_accuracies["Scree with grass"]+    class_accuracies["Water"]+   class_accuracies["Bedrockwith grass"])
rare_cls_acc =   1/2* (class_accuracies["Large blocks"]+   class_accuracies["Large blocks with grass"])
metrics = {
            'test_miou' : np.round ( mean_iou,3), 
            'test_macc':  np.round (mean_acc,3), 
            'test_oacc':  np.round (overall_acc,3),
            'frequent_cls_acc':np.round(freq_cls_acc,3),
            'common_cls_acc': np.round(common_cls_acc,3),
            'rare_cls_acc': np.round(rare_cls_acc,3),
            'test_acc' :  class_accuracies,
    }
pprint(metrics)


mean_acc, mean_iou, overall_acc 
expert 0
{'common_cls_acc': 0.251,
 'frequent_cls_acc': 0.783,
 'rare_cls_acc': 0.011,
 'test_acc': {'Background': 0.0,
              'Bedrock': 0.714,
              'Bedrockwith grass': 0.013,
              'Forest': 0.845,
              'Glacier': 0.822,
              'Large blocks': 0.022,
              'Large blocks with grass': 0.0,
              'Scree': 0.751,
              'Scree with grass': 0.197,
              'Water': 0.543},
 'test_macc': 0.391,
 'test_miou': 0.326,
 'test_oacc': 0.75}
expert 1
{'common_cls_acc': 0.773,
 'frequent_cls_acc': 0.031,
 'rare_cls_acc': 0.346,
 'test_acc': {'Background': 0.011,
              'Bedrock': 0.002,
              'Bedrockwith grass': 0.573,
              'Forest': 0.044,
              'Glacier': 0.072,
              'Large blocks': 0.693,
              'Large blocks with grass': 0.0,
              'Scree': 0.006,
              'Scree with grass': 0.832,
              'Water': 0.914},
 'test_macc': 0.315